<a href="https://colab.research.google.com/github/betty6you/Jing_INFO5731_Spring2020/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [8]:
# Write your code here
# 10 fold cross validation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import svm

In [14]:
# reading training data

data= open('stsa-train1.txt').read()
labels,texts=[],[]
for line in data.split("\n"):
  content=line.split( )  
  labels.append(content[0])
  texts.append(" ".join(content[1:]))

# create a dataframe using texts and labels
trainDF=pd.DataFrame()
trainDF['text']=texts
trainDF['label']=labels
trainDF

,text,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


In [15]:
import nltk
import re
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob
nltk.download('wordnet')
st = PorterStemmer()
nltk.download('punkt')
stop = stopwords.words('english')
# data clean
trainDF['Lowercase'] = trainDF['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
trainDF['Removepunct'] = trainDF['Lowercase'].str.replace('[^\w\s]','')
trainDF['Removespecialchar'] = trainDF['Removepunct'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
trainDF['Removestopwords'] = trainDF['Removespecialchar'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
trainDF['Tokenization'] = trainDF['Removestopwords'].apply(lambda x: TextBlob(x).words)
trainDF['Lemmatization'] = trainDF['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))
trainDF

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,label,Lowercase,Removepunct,Removespecialchar,Removestopwords,Tokenization,Lemmatization
0,"a stirring , funny and finally transporting re...",1,"a stirring , funny and finally transporting re...",a stirring funny and finally transporting rei...,a stirring funny and finally transporting rei...,stirring funny finally transporting reimaginin...,"[stirring, funny, finally, transporting, reima...",stirring funny finally transporting reimaginin...
1,apparently reassembled from the cutting-room f...,0,apparently reassembled from the cutting-room f...,apparently reassembled from the cuttingroom fl...,apparently reassembled from the cuttingroom fl...,apparently reassembled cuttingroom floor given...,"[apparently, reassembled, cuttingroom, floor, ...",apparently reassembled cuttingroom floor given...
2,they presume their audience wo n't sit still f...,0,they presume their audience wo n't sit still f...,they presume their audience wo nt sit still fo...,they presume their audience wo nt sit still fo...,presume audience wo nt sit still sociology les...,"[presume, audience, wo, nt, sit, still, sociol...",presume audience wo nt sit still sociology les...
3,this is a visually stunning rumination on love...,1,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,visually stunning rumination love memory histo...,"[visually, stunning, rumination, love, memory,...",visually stunning rumination love memory histo...
4,jonathan parker 's bartleby should have been t...,1,jonathan parker 's bartleby should have been t...,jonathan parker s bartleby should have been th...,jonathan parker s bartleby should have been th...,jonathan parker bartleby beallendall modernoff...,"[jonathan, parker, bartleby, beallendall, mode...",jonathan parker bartleby beallendall modernoff...
...,...,...,...,...,...,...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1,"painful , horrifying and oppressively tragic ,...",painful horrifying and oppressively tragic t...,painful horrifying and oppressively tragic t...,painful horrifying oppressively tragic film mi...,"[painful, horrifying, oppressively, tragic, fi...",painful horrifying oppressively tragic film mi...
6916,take care is nicely performed by a quintet of ...,0,take care is nicely performed by a quintet of ...,take care is nicely performed by a quintet of ...,take care is nicely performed by a quintet of ...,take care nicely performed quintet actresses n...,"[take, care, nicely, performed, quintet, actre...",take care nicely performed quintet actress non...
6917,"the script covers huge , heavy topics in a bla...",0,"the script covers huge , heavy topics in a bla...",the script covers huge heavy topics in a blan...,the script covers huge heavy topics in a blan...,script covers huge heavy topics bland surfacey...,"[script, covers, huge, heavy, topics, bland, s...",script cover huge heavy topic bland surfacey w...
6918,a seriously bad film with seriously warped log...,0,a seriously bad film with seriously warped log...,a seriously bad film with seriously warped log...,a seriously bad film with seriously warped log...,seriously bad film seriously warped logic writ...,"[seriously, bad, film, seriously, warped, logi...",seriously bad film seriously warped logic writ...


In [18]:
# reading test data
testdata= open('stsa-test.txt').read()
testlabels,testtexts=[],[]
for line in testdata.split("\n"):
  content=line.split( )  
  testlabels.append(content[0])
  testtexts.append(" ".join(content[1:]))

# create a dataframe using texts and labels\
testDF=pd.DataFrame()
testDF['text']=testtexts
testDF['label']=testlabels
testDF

,text,label
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1
...,...,...
1816,"an often-deadly boring , strange reading of a ...",0
1817,the problem with concept films is that if the ...,0
1818,"safe conduct , however ambitious and well-inte...",0
1819,"a film made with as little wit , interest , an...",0


In [19]:
# test data cleaning
testDF['Lowercase'] = testDF['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
testDF['Removepunct'] = testDF['Lowercase'].str.replace('[^\w\s]','')
testDF['Removespecialchar'] = testDF['Removepunct'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
testDF['Removestopwords'] = testDF['Removespecialchar'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
testDF['Tokenization'] = testDF['Removestopwords'].apply(lambda x: TextBlob(x).words)
testDF['Lemmatization'] = testDF['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))
testDF

,text,label,Lowercase,Removepunct,Removespecialchar,Removestopwords,Tokenization,Lemmatization
0,"no movement , no yuks , not much of anything .",0,"no movement , no yuks , not much of anything .",no movement no yuks not much of anything,no movement no yuks not much of anything,movement yuks much anything,"[movement, yuks, much, anything]",movement yuks much anything
1,"a gob of drivel so sickly sweet , even the eag...",0,"a gob of drivel so sickly sweet , even the eag...",a gob of drivel so sickly sweet even the eage...,a gob of drivel so sickly sweet even the eage...,gob drivel sickly sweet even eager consumers m...,"[gob, drivel, sickly, sweet, even, eager, cons...",gob drivel sickly sweet even eager consumer mo...
2,"gangs of new york is an unapologetic mess , wh...",0,"gangs of new york is an unapologetic mess , wh...",gangs of new york is an unapologetic mess who...,gangs of new york is an unapologetic mess who...,gangs new york unapologetic mess whose saving ...,"[gangs, new, york, unapologetic, mess, whose, ...",gang new york unapologetic mess whose saving g...
3,"we never really feel involved with the story ,...",0,"we never really feel involved with the story ,...",we never really feel involved with the story ...,we never really feel involved with the story ...,never really feel involved story ideas remain ...,"[never, really, feel, involved, story, ideas, ...",never really feel involved story idea remain a...
4,this is one of polanski 's best films .,1,this is one of polanski 's best films .,this is one of polanski s best films,this is one of polanski s best films,one polanski best films,"[one, polanski, best, films]",one polanski best film
...,...,...,...,...,...,...,...,...
1816,"an often-deadly boring , strange reading of a ...",0,"an often-deadly boring , strange reading of a ...",an oftendeadly boring strange reading of a cl...,an oftendeadly boring strange reading of a cl...,oftendeadly boring strange reading classic who...,"[oftendeadly, boring, strange, reading, classi...",oftendeadly boring strange reading classic who...
1817,the problem with concept films is that if the ...,0,the problem with concept films is that if the ...,the problem with concept films is that if the ...,the problem with concept films is that if the ...,problem concept films concept poor one saving ...,"[problem, concept, films, concept, poor, one, ...",problem concept film concept poor one saving m...
1818,"safe conduct , however ambitious and well-inte...",0,"safe conduct , however ambitious and well-inte...",safe conduct however ambitious and wellintent...,safe conduct however ambitious and wellintent...,safe conduct however ambitious wellintentioned...,"[safe, conduct, however, ambitious, wellintent...",safe conduct however ambitious wellintentioned...
1819,"a film made with as little wit , interest , an...",0,"a film made with as little wit , interest , an...",a film made with as little wit interest and ...,a film made with as little wit interest and ...,film made little wit interest professionalism ...,"[film, made, little, wit, interest, profession...",film made little wit interest professionalism ...


In [23]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word')
tfidf_vect.fit(trainDF['Lemmatization'])
x_tfidf =  tfidf_vect.transform(trainDF['Lemmatization'])
vect_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vect_test.fit(testDF['Lemmatization'])
xtest = vect_test.transform(testDF['Lemmatization'])
test_y = testDF['label']

In [33]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn import metrics
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x_tfidf,trainDF['label'].values,test_size=0.2)
print(train_y)
from sklearn import metrics
def get_metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1

import numpy as np
from sklearn.model_selection import cross_val_score, KFold
def cross_validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val

['0' '0' '0' ... '1' '1' '0']


In [36]:
#multinomialNB model
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

naive_bayes_model = naive_bayes.MultinomialNB()
naive_bayes_model.fit(train_x, train_y)
naive_bayes_predictions_validation_data = naive_bayes_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# cross validation score
nb_scores_mean = cross_validation_score(naive_bayes_model, valid_x, valid_y)
print('Navie Bayes Cross Validation Score is {0}'.format(nb_scores_mean))
#model performance evaluation
naive_bayes_predictions_test_data = naive_bayes_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7832369942196532
Precision is 0.7832369942196532
Recall is 0.7832369942196532
F1 is 0.7832369942196532
Navie Bayes Cross Validation Score is 0.735564591804817
Accuracy is 0.7880285557386052
Precision is 0.7880285557386052
Recall is 0.7880285557386052
F1 is 0.7880285557386053


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [39]:
# SVM model
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_predictions_validation_data = svm_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# Cross validation score
svm_scores_mean = cross_validation_score(svm_model, valid_x, valid_y)
print('SVM Cross Validation Score is {0}'.format(svm_scores_mean))
svm_predictions_test_data = svm_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.7796242774566474
Precision is 0.7796242774566474
Recall is 0.7796242774566474
F1 is 0.7796242774566473
SVM Cross Validation Score is 0.7081169846731311
Accuracy is 0.7951674903898956
Precision is 0.7951674903898956
Recall is 0.7951674903898956
F1 is 0.7951674903898956


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [42]:
# KNN model
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 15)
knn_model.fit(train_x, train_y)
knn_predictions_valid_data = knn_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# Cross validation score
knn_scores_mean = cross_validation_score(knn_model, valid_x, valid_y)
print('KNN Cross Validation Score is {0}'.format(knn_scores_mean))
# KNN performance evaluation
knn_predictions_test_data = knn_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.7239884393063584
Precision is 0.7239884393063584
Recall is 0.7239884393063584
F1 is 0.7239884393063584
KNN Cross Validation Score is 0.6856949223230112
Accuracy is 0.729818780889621
Precision is 0.729818780889621
Recall is 0.729818780889621
F1 is 0.729818780889621


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [43]:
#Decision tree model
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(train_x, train_y)
dt_predictions_valid_data = dt_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# Cross validation score
dt_scores_mean = cross_validation_score(dt_model, valid_x, valid_y)
print('Decision Tree Cross Validation Score is {0}'.format(dt_scores_mean))
# Decision tree performance evaluation
dt_predictions_test_data = dt_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.6763005780346821
Precision is 0.6763005780346821
Recall is 0.6763005780346821
F1 is 0.6763005780346821
Decision Tree Cross Validation Score is 0.5852830778855177
Accuracy is 0.6540362438220758
Precision is 0.6540362438220758
Recall is 0.6540362438220758
F1 is 0.6540362438220758


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [44]:
# Random forest model
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_x, train_y)
rf_predictions_valid_data = rf_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# Cross validation score
rf_scores_mean = cross_validation_score(rf_model, valid_x, valid_y)
print('Random Forest Cross Validation Score is {0}'.format(rf_scores_mean))
# model performance evaluation
rf_predictions_test_data = rf_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.7333815028901735
Precision is 0.7333815028901735
Recall is 0.7333815028901735
F1 is 0.7333815028901733
Random Forest Cross Validation Score is 0.651089563132103
Accuracy is 0.7413509060955519
Precision is 0.7413509060955519
Recall is 0.7413509060955519
F1 is 0.7413509060955519


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [45]:
# XG Boost validation Matrix
from xgboost import XGBClassifier
xg_model = XGBClassifier()
xg_model.fit(train_x, train_y)
xg_predictions_valid_data = xg_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
# Cross validation score
xg_scores_mean = cross_validation_score(XGBClassifier(), valid_x, valid_y)
print('XG Boost Cross Validation Score is {0}'.format(xg_scores_mean))
# model evaluation
xg_predictions_test_data = xg_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.6278901734104047
Precision is 0.6278901734104047
Recall is 0.6278901734104047
F1 is 0.6278901734104047
XG Boost Cross Validation Score is 0.6011156292357418
Accuracy is 0.6331685886875343
Precision is 0.6331685886875343
Recall is 0.6331685886875343
F1 is 0.6331685886875343
